# Content Based Filtering

In [1]:
import os
import sys

sys.path.append("/Users/eric/Code/recommender-system/src")

In [2]:
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

from data.db import Recommendations
from data.db import insert, setup

In [3]:
movies = pd.read_csv("../datasets/movielens/ml-20m/movies.csv")

## TF-IDF on genres

In [5]:
vec = TfidfVectorizer()
tfidf_matrix = vec.fit_transform(movies["genres"])

cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
idx_mapper = pd.Series(movies.index, index=movies['movieId'])

def predict(movie_id):
    index = idx_mapper[movie_id]
    sim_scores = list(enumerate(cosine_sim[index]))
    sim_scores_sorted = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    movie_ids = idx_mapper.iloc[[i[0] for i in sim_scores_sorted]].index.tolist()
    scores = [i[1] for i in sim_scores_sorted]

    return list(zip(movie_ids, scores))[:50]

In [6]:
%%time

to_insert = []

for index in idx_mapper.index.tolist():
    for r in predict(index):
        if index == r[0]:
            continue
        to_insert.append(Recommendations(**{"engine_name": "TfidfGenres",
                                            "source_product_id": index,
                                            "recommended_product_id": r[0],
                                            "score": r[1]}))

CPU times: user 18min 28s, sys: 25.7 s, total: 18min 53s
Wall time: 18min 56s


In [8]:
insert(to_insert)

In [9]:
import pickle

In [ ]:
with open("cosine.pkl", "w") as output:
    pickle.dump(to_insert, output)